In [2]:
import time
import numpy as np
import random
import os

from PPO.ppo import Model
from game.game import Game
from PBT.population import Population
from learning.util import make_load_hanabi_fn, get_env_spec
import tensorflow as tf



In [3]:
rewards_weights_base = {'play0' : 1, 'play1' : 3, 'play2' : 9, 'play3' : 27, 'play4' : 81,
                  'baseline' :3,
                  'discard_last_copy' : -100, 'discard_extra' : 0.5,
                  'hint_last_copy' : 0.2, 'hint_penalty' : 0.1,  'hint_playable' : 0.2,
                  'use_hamming' : True, 'loose_life' : -50}

env_config = {'environment_name' : 'Hanabi-Small', 'num_players' : 2, 
              'use_custom_rewards' : True, 'open_hands' : False}

model_config_base = {'scope' : '', 'nenvs'  : 16, 
           'fc_input_layers' : [128], 'noisy_fc' : False, 'v_net' : 'shared',
           'gamma' : 1, 'ent_coef' : 0.01, 'vf_coef' : 1, 'cliprange' : 0.2,
           'max_grad_norm' : None, 'k' : 24, 
           'lr' : 1e-4, 'anneal_lr' : False,
           'normalize_advs': True, 'layer_norm' : False,
           }


# these attributes of the model will evolove
random_attributes = {'lr' : lambda : np.random.uniform(1e-5, 1e-3),
                 'k' : lambda : np.random.randint(2, 36), 
                 'cliprange' : lambda : np.random.uniform(0.05, 0.4),
                 'vf_coef' : lambda : np.random.uniform(0.1, 1.7), 
                 'ent_coef' : lambda : np.random.uniform(0.001, 0.075),
                 'nsteps' : lambda : np.random.randint(2, 40)}

root_folder = './experiments/PBT/'

In [4]:
# setting parameters

nmodels =  20
n_to_evolve = 5

mutation_prob = 0.5
ts_per_epoch = 600000
evaluate_last_updates = 50

population_name = 'pbt_test'
save_every = 20


In [5]:
# create pool    
load_env_fn = make_load_hanabi_fn(env_config)
env = load_env_fn()
nobs, nactions, nplayers = get_env_spec(env_config)
nenvs = model_config_base['nenvs']


tf.reset_default_graph()
sess = tf.Session()

game  = Game(nplayers, nenvs, load_env_fn, wait_rewards = True)
population = Population(nactions, nobs, 2, sess, nmodels,
                        model_config_base, rewards_weights_base, random_attributes,
                       folder = root_folder, name = population_name)


GAME PARAMETERS: 
  observation length = 171
  number of actions = 11
  number of players = 2
normal playa
normal playa



Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Use `tf.random.categorical` instead.


Instructions for updating:
Use `tf.random.categorical` instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Could not load model "agent0" at ./experiments/PBT/pbt_test/agent0/model/
Could not load model "agent1" at ./experiments/PBT/pbt_test/agent1/model/
Could not load model "agent2" at ./experiments/PBT/pbt_test/agent2/model/
Could not load model "agent3" at ./experiments/PBT/pbt_test/agent3/model/
Could not load model "agent4" at ./experiments/PBT/pbt_test/agent4/model/
Could not load model "agent5" at ./experiments/PBT/pbt_test/agent5/model/
Could not load model "agent6" at ./experiments/PBT/pbt_test/agent6/model/
Could not load model "agent7" at ./experiments/PBT/pbt_test/agent7/model/
Could not load model "agent8" at ./experiments/PBT/pbt_test/agent8/model/
Could not load model "agent9" at ./experiments/PBT/pbt_test/agent9/model/
Could not load model "agent10" at ./experiments/PBT/pbt_test/agent10/model/
Could not load model "agent11" at ./experiments/PBT/pbt_test/agent11/model/
Could not load model "agent12" at ./experiments/PBT/pbt_test/agent12/model/
Could not load model "agent13" a

In [ ]:
for epoch in range(100000):
    population.run_epoch(game, ts_per_epoch, n_to_evolve, 
                         episodes_to_train = 90, nsteps_to_train = 16,
                         save_every = 500000, summary_every = 20000)